# Applied Deep Learning Tutorial 
contact: Mark.schutera@kit.edu


# Segmentation with U-Net

## Introduction
This tutorial presents a segmentation use-case based on U-Net network (see paper for reference [paper](https://arxiv.org/pdf/1505.04597.pdf)).
First, the theoretical background behind the architecture is discussed. Subsequently you will be guided through the implementation process of the neural network within this Jupyter Notebook. The use-case is based on the initial paper and focuses on cell boundary segmentation, see Fig.1.
<img src="graphics/unet_fig_1.png" width="700"><br>
<center> Figure 1: Right: input image | Left: Prediction of the cell boundaries </center>

## Motivation

While segmentation networks and convolutional neural networks often need a very large amount of training data in order to converge, it is necessary to deploy the training data more efficiently. In segmentation, objects need to be localized and classified. This requires improved semantic understanding. 
The U-Net achieves sample efficiency while capturing both class affiliations and localization performance.  

## U-Net Architecture

<img src="graphics/unet_fig_2.png" width="700"><br>
<center> Figure 2: U-Net architecture </center>

In Fig.2, the architecture of the U-Net is shown. The name is derived from the U-shaped architecture of the downsampling and the upsampling path. The input to the U-Net is an image tile. An image is processed one tile after another. In this manner, the image can be of an arbitrarily large size and for example and its size is not linked to the memory capabilities of your GPU. An example of such a tile can be seen in Fig.3 below.
<img src="graphics/unet_fig_3.png" width="300">


<center> Figure 3: Tile of an image </center>
A tile of an input image is shown. The convolution has no padding, thus the yellow box, the center of the tile, gets a segment prediction output while the blue region is the region which is fed into the neural network. 

The convolutions marked with the blue arrows within Fig.2 process the input by building feature maps of the shown size. These convolution a ReLu function is followed. After the first 3 convolutions a feature map of the size 568x568x64 is received. Thereafter max-pooling operation (2x2 with a stride of 2), marked with red arrows in the image, downsample the image size but simultaneously doubles the size of the feature map.<br>
This is repeated until we get a single vector with a high resolution feature map. Thus this contracting path reduces the information of the location by simultaneously increase the context information (features). From this high resolution feature vector, now a high resolution segmentation-map with a high accuracy of the location is built. This is done by expand the vector with up-sampling, due up-convolution and with, and this is the novelty, concatenate the high resolution feature map from the contracting path. This is shown with the grey arrows seen in the Fig.2. Furthermore after every upconcolution a ReLU operation is appended. 
With this combination of knotting the upconcolution of the high resolution feature map with the localization information, it is not getting loss as it is the case with just do an "autoencoding". <br>
Due to the aforementioned loss of information with the nonpadded convolution, the feature map needs to be cropped.
The final layer consists out of a 1x1 convolution which maps each 64 component feature vector to the desired number of classes. And thus devide the image into the requested segments. 

## Implementing the code 

We will orientate with this tutorial on the implemtation of the code from [this](https://github.com/zhixuhao/unet) github repository.
Due to the implementation in Jupyter Notebook we will write the whole code in one template. Instead of different python documents we will use classes.
The original repository consists out of three python documents. To work only with this notebooky every function is integrated here, thus you don't have to switch documents or import other python-files.


In [ ]:
# If you do not have the data yet, you can download it here:
# https://github.com/zhixuhao/unet/tree/master/data/membrane
# Now go to the folder in which this jupyter Notebook ist located and running
# and copy the <<data>> folder there and renaming either the folders according
# to the Path written below or change the path written below according to
# your folder-names

When you run the Code below and you set the Path accordingly, an image and its label should be shown.

In [ ]:
##########################################################
#           In this box some code must be added          #
##########################################################
from IPython.display import Image

# We want to see one of the training data with its label
# Change the folders or the code that the path points to your data
training_images = "data/membrane/train/image/"
training_labels = "data/membrane/train/label/"

# We use the function Image(file_name) to print the image '0.png' and the label 'o.png'
display(Image(training_images + "0.png"))
display(Image(training_labels + "0.png"))

### Buidling the Unet Architecture

In the next box all libraries and packages we will need for the U-Net and its data prepration will be imported. 
Because Keras is a part of tensorflow and is installed within, it comes preinstalled with tensorflow <br>
"tensorflow.keras"

If something went wrong with importing the packages, you can try to restart your kernel.

In [ ]:
from __future__ import print_function

import glob
import os

import numpy as np
import skimage.io as io  # pip install scikit-image pip install scipy

# if This Error occurs:
# <<ImportError: cannot import name 'rgb2gray'>>
# restart your kernel
import skimage.transform as trans
import tensorflow as tf
from keras import Sequential
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.utils import image_dataset_from_directory

To build the archtecture which is shown in fig.1 keras is a very intuitive and simply way to do so. We will slithly adapt the archtecture from the original one like seen in fig.3. To build this neural net we do need the following functions:
- 2D Convolution
- MaxPooling 
- Concatenation

We will add an Dropout after convolution 4 and 5,  please explain what could be the purpose to add those two layers?






Let's start coding the Network with the following functions:

- Conv2D(filters/feature maps, kernel size, activation, padding, kernel_initializer)(input Data)
- MaxPooling2D(pool_size = (x,x))(input Data)
- Dropout(dropout rate)(input Data)
- concatenate([input Data 1, Input Data 2], axis= x)
- Conv2D()(UpSampling2D(size=(x,x))(Input Data)

As activation function we take 'relu' (Rectified Linear Unit).
As padding we chose 'same', thus despite convolution we get an output of the same size as the input.
The kernel initializer defines how we want to set the initial weights of this Keras layer. For example:
- Zeros: initialize with a tensor of zeros
- Ones: initialize with a tensor of ones
- RandomNormal: initialize the tensor with a normal distribution
- TruncatedNormal: initialize the tensor with a truncated normal distribution, where all values more than two standard deviations from its mean are neglected
- he_normal: a zero mean Gaussian distribution with a standard deviation of \sqrt{(2/n)}, where n is the number of inputs/connections of our weight tensor <br>

Which initialization do you think is beneficial?




When the architecture is configured correctly we have to define the inputs and outputs of our Neural Network with the method: <br> Model(inputs= ___ , outputs= __).
Thereafter we need to compile the whole architecture for training. Therefore we need to define the optimizer, the learning rate, the loss function and we can determine on which metrics we want to evaluate our model. 

List 3 loss functions and 3 optimizers and explain them:


Choose one of these optimizers you thing fit best for your U-Net and experiment with the learning rate.
The return of our function should be the model.


## The Dice Coefficient Loss

In this section we will implement the Dice Coefficient loss, so we can use it with our own model.
The Dice Coefficient, also known as Sørensen–Dice coefficient compares the similarity of two quantities. When working with neural network we want to compare the prediction of our network and the ground truth. The dice coefficient can be interpreted as Intersection over union, this makes it intuitive to interpret and a direct loss function for segmentation errors.
<br>
The equation of the dice loss is given with:
\begin{align}
    DSC = \frac{2*|X \cap Y|}{|X|+|Y|}
\end{align}

Before coding this equation you may think about:
   - do I need the absolute values?
   - how can i easily calculate the intersection, assuming that the ground truth is a one-hot vector:
    \begin{align}
        y_{true} = \begin{bmatrix}
        0.0 &  0.0 &  0.0 &  1.0
        \end{bmatrix}
    \end{align}
    \begin{align}
        y_{pred} = \begin{bmatrix}
        0.1 &  0.2 &  0.0 &  0.7
        \end{bmatrix}
    \end{align}
<br>

Now start with the coding, following these steps: 
- First we want to flatten our input to the loss y_true and y_pred
- Calculate the counter with its intersection
- Calulate the denominator, (you may want to add a small epsilon to it)
- Obtain the coefficient

Because we work here with the backend of keras, we need to do our calculations with K.operation.
K is the backend we imported above with "from tensorflow.keras import backend as K".
Here a list of some operations is listed, if you need more, just google them:
- sum: K.sum()
- flatten: K.flatten()
- mean: K.mean()
- reshape: K.reshape()
- dot: K.dot()
- epsilon: K.epsilon()

In [ ]:
def dice_coef(y_true, y_pred):
    y_true_flatten = 
    y_pred_flatten = 
    intersection = 
    nominator = 
    denominator = 
    coef = 
    return coef

Why it is advised to add an epsilon to your denominator?


In [ ]:

def dice_coef_loss(y_true, y_pred):
    dice_coef_loss = 
    return dice_coef_loss

In [ ]:
##########################################################
#           In this box some code must be added          #
##########################################################


def unet(
    pretrained_weights=None, input_size=(256, 256, 1)
):  # the values given for the parameters are the default values
    inputs = Input(input_size)
    # complete the missing layers (your may use the lines below the ones to fill in, if you are stuck)
    conv1 = Conv2D(64, 3, activation=__, padding=__, kernel_initializer=__)(__)
    conv1 = Conv2D(64, 3, activation=__, padding=__, kernel_initializer=__)(__)
    # Maxpooling with 2x2
    pool1 = MaxPooling2D(pool_size=(__))(__)
    conv2 = Conv2D(__, 3, activation=__, padding=__, kernel_initializer=__)(__)
    conv2 = Conv2D(__, 3, activation=__, padding=__, kernel_initializer=__)(__)
    pool2 = MaxPooling2D(pool_size=(2, 2))(__)
    conv3 = Conv2D(__, 3, activation=__, padding=__, kernel_initializer=__)(__)

    ################################################################################################
    conv3 = Conv2D(
        256, 3, activation="relu", padding="same", kernel_initializer="he_normal"
    )(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(
        512, 3, activation="relu", padding="same", kernel_initializer="he_normal"
    )(pool3)
    conv4 = Conv2D(
        512, 3, activation="relu", padding="same", kernel_initializer="he_normal"
    )(conv4)
    ################################################################################################

    # Here add a Dropout of 0.5
    drop4 = __
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(
        1024, 3, activation="relu", padding="same", kernel_initializer="he_normal"
    )(pool4)
    conv5 = Conv2D(
        1024, 3, activation="relu", padding="same", kernel_initializer="he_normal"
    )(conv5)
    drop5 = Dropout(0.5)(conv5)
    ################################################################################################
    # Here we will start with the upconvolution
    ################################################################################################
    up6 = Conv2D(
        512, 2, activation="relu", padding="same", kernel_initializer="he_normal"
    )(
        # ____ insert the input of this first upsampling layer with using the function:
        # << UpSampling2D()>> with the same size as the MaxPooling layers and as input the according layer.
    )
    # Because we now have the output of our UpConvolution but also we need the spatial information from the
    # convolutional layers and MaxPooling we have to merge these two inputs with the function <<concatenate>>
    # Figurativly speaking merge the gray and green arrows from Fig. 2
    merge6 = concatenate([___, __], axis=3)
    conv6 = Conv2D(
        512, 3, activation="relu", padding="same", kernel_initializer="he_normal"
    )(merge6)
    conv6 = Conv2D(
        512, 3, activation="relu", padding="same", kernel_initializer="he_normal"
    )(conv6)

    up7 = Conv2D(
        256, 2, activation="relu", padding="same", kernel_initializer="he_normal"
    )(UpSampling2D(size=(2, 2))(conv6))
    merge7 = concatenate([conv3, up7], axis=3)
    conv7 = Conv2D(
        256, 3, activation="relu", padding="same", kernel_initializer="he_normal"
    )(merge7)
    conv7 = Conv2D(
        256, 3, activation="relu", padding="same", kernel_initializer="he_normal"
    )(conv7)

    up8 = Conv2D(
        128, 2, activation="relu", padding="same", kernel_initializer="he_normal"
    )(UpSampling2D(size=(2, 2))(conv7))
    merge8 = concatenate([conv2, up8], axis=3)
    conv8 = Conv2D(
        128, 3, activation="relu", padding="same", kernel_initializer="he_normal"
    )(merge8)
    conv8 = Conv2D(
        128, 3, activation="relu", padding="same", kernel_initializer="he_normal"
    )(conv8)

    up9 = Conv2D(
        64, 2, activation="relu", padding="same", kernel_initializer="he_normal"
    )(UpSampling2D(size=(2, 2))(conv8))
    merge9 = concatenate([conv1, up9], axis=3)
    conv9 = Conv2D(
        64, 3, activation="relu", padding="same", kernel_initializer="he_normal"
    )(merge9)
    conv9 = Conv2D(
        64, 3, activation="relu", padding="same", kernel_initializer="he_normal"
    )(conv9)
    conv9 = Conv2D(
        2, 3, activation="relu", padding="same", kernel_initializer="he_normal"
    )(conv9)

    # In the last layer we chose the <sigmoid> as activation function because the output has to be between zero and one.
    # We normalized out pixle values by deviding through 255.
    # For our example we have only one class and thus only needing
    # zero being white (our predicted object-segment) and 1 beeing black.
    # but if we have more that one class the different colors will also lie inbetween 0 and 1 and every
    # prediction is labeld with the appropriate number.

    # Choose a function for your last layer which fullfill the requirements for classification at its best
    conv10 = Conv2D(1, 1, activation=__)(conv9)

    # With the following method <Model> we define out input and output of or Neural Network
    # please complete:
    model = Model(inputs=inputs, outputs=conv10)

    # with Model.compile we configure and prepare our network for training. Thus we define our optimizer for
    # learning with it's learning rate, the loss function and the metrics.
    # Choose a loss you think fits best for this kind of task and also do so for the optimizer
    # if your own coded optimizer and loss should be used you have to implement them accoridingly
    model.compile(optimizer=__(learning_rate=__), loss=___, metrics=["accuracy"])

    # model.summary()

    # this part we need for resume to a training or printing test, results for an already trained network, be careful
    # that the input_size you defined for the model match with the one you defined for the model you take the pretrained
    # weights from!
    # Here you can play with some provided networks for more difficult tasks and do not train them expensivly on your own
    # Computer
    if pretrained_weights:
        model.load_weights(pretrained_weights)

    return model

### Preparing the Data

So far we build the architecture of our neural net and the data, consists out of images and labels. To train the neural network in the next step the data has to be prepared.

#### Adjusting and Augmenting the Data

Before we feed the data to our neural network, it has to be adjusted properly. 
The first step always helps is to normalize our input data. Important is to normalize both the labels and the images, so that the neural network does not learn something wrong, warped or even nothing at all.
In this tutorial we got the binary case and therefor the images and masks just need to be normalized.
This is done by dividing through 255. 
The way we augment data in modern tensorflow is to use a Sequential model with augmentation layers, that return the input tensor in an augmented way. By using these layers we never need to convert our input tensors we created into a different format and can just later on run the input tensors and the labeled output tensors through the same model and get our augmented pair. 
It is very important to use the same seed in both `` adjust_training_images `` and `` adjust_training_masks ``, because if we for example rotate an image, the corresponding laber has to be rotated in the same way.

In [ ]:
##########################################################
#           In this box some code must be added          #
##########################################################

def adjust_training_images(
    rotation_range: float,
    width_shift_range: float,
    height_shift_range: float,
    zoom_range: float,
    resize_height: int,
    resize_width: int,
    flip="horizontal",
    interpolation="nearest",
    seed=1,
):

    sequential_augmentation = Sequential(
        [
            tf.keras.layers.Rescaling(scale= __ / ___ ),
            tf.keras.layers.RandomRotation(
                factor=rotation_range, interpolation=interpolation, seed=seed
            ),
            tf.keras.layers.RandomZoom(
                height_factor=zoom_range,
                width_factor=zoom_range,
                fill_mode="reflect",
                interpolation=interpolation,
                seed=seed,
            ),
            tf.keras.layers.RandomFlip(mode=flip, seed=seed),
            tf.keras.layers.RandomHeight(
                factor=height_shift_range, interpolation=interpolation, seed=seed
            ),
            tf.keras.layers.RandomWidth(
                factor=width_shift_range, interpolation=interpolation, seed=seed
            ),
            tf.keras.layers.Resizing(
                resize_height,
                resize_width,
                interpolation="bilinear",
                crop_to_aspect_ratio=False,
            ),
        ]
    )
    return sequential_augmentation


def adjust_training_masks(
    rotation_range: float,
    width_shift_range: float,
    height_shift_range: float,
    zoom_range: float,
    resize_height: int,
    resize_width: int,
    flip="horizontal",
    interpolation="nearest",
    seed=1,
):

    sequential_augmentation = Sequential(
        [
            tf.keras.layers.Rescaling(scale= __ / ___ ),
            tf.keras.layers.RandomRotation(
                factor=rotation_range, interpolation=interpolation, seed=seed
            ),
            tf.keras.layers.RandomZoom(
                height_factor=zoom_range,
                width_factor=zoom_range,
                fill_mode="reflect",
                interpolation=interpolation,
                seed=seed,
            ),
            tf.keras.layers.RandomFlip(mode=flip, seed=seed),
            tf.keras.layers.RandomHeight(
                factor=height_shift_range, interpolation=interpolation, seed=seed
            ),
            tf.keras.layers.RandomWidth(
                factor=width_shift_range, interpolation=interpolation, seed=seed
            ),
            tf.keras.layers.Resizing(
                resize_height,
                resize_width,
                interpolation="bilinear",
                crop_to_aspect_ratio=False,
            ),
        ]
    )
    return sequential_augmentation


rescale_data = Sequential([tf.keras.layers.Rescaling(scale= __ / ___ )])

#### The training Generator

The next function will be the ``trainGenerator``.
Because deep learning task only get to learn something if they are trained with a lot of data, we have to provide it. But with the data we downloaded, there are only 30 images in the folder ``train/``. For a deep learning trask that is way to little. Therefor we will generating our own data with data augmentation. This function will use the data augmentation layers created above on the given images with layers like RandomRotation, RandomFlip and RandomZoom etc. Here the most important is that labels and images are transformed together with the same transformation functions. How good such an augmentation can replace more recordings strongly depends from the data on which we will learn. But for the given task the data augmentation works really good.
The function used here to generate the Data is the image_dataset_from_directory provided within Keras. The documentations for more information can be found here 
https://keras.io/preprocessing/image/.

To use the image augmentation function from above, we map the output onto the augmentation function. 

Why is it do important that you will chose the same seed for the mask and the image itself and what does seed stand for?



In [ ]:
##########################################################
#           In this box some code must be added          #
##########################################################
def trainGenerator(
    batch_size,
    train_path,
    image_folder,
    mask_folder,
    aug_dict,
    image_color_mode="grayscale",
    mask_color_mode="grayscale",
    target_size=(256, 256),
    seed=1,
):
    """
    can generate image and mask at the same time
    use the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same
    if you want to visualize the results of generator, set save_to_dir = "your path"
    """
    # create two augmentation sequences with the same seed, to augment the images and the labels in the same way.
    # This is crucial because otherwise the labels wouldn't fit the images anymore.
    image_augmentation = adjust_training_images(**aug_dict)
    mask_augmentation = adjust_training_masks(**aug_dict)

    # Here we create a generator object and the map the augmentation and rescaling Sequences on the generators.
    image_generator = (
        image_dataset_from_directory(
            train_path + "/" + image_folder,
            labels=None,
            label_mode="binary",
            color_mode=image_color_mode,
            image_size=target_size,
            batch_size=batch_size,
            seed=seed,
        )
        .repeat()
        .map(lambda x: ___ (x)) # What function do we want to map here?
    )

    mask_generator = (
        image_dataset_from_directory(
            train_path + "/" + mask_folder,
            labels=None,
            label_mode="binary",
            color_mode=mask_color_mode,
            image_size=target_size,
            batch_size=batch_size,
            seed=seed,
        )
        .repeat()
        .map(lambda x: ___ (x)) # What function do we want to map here?
    )

    train_generator = zip(image_generator, mask_generator)
    for (img, mask) in train_generator:
        yield (img, mask)



# to understand what yield is: https://stackoverflow.com/questions/231767/what-does-the-yield-keyword-do

#### The Testing Generator

The testGenerator function will generate your test data on the images provided within the test folder. 
It will first read the images from the test-folder as gray images normalize them and resize them like defined in with the parameter target_size. Depending on the amount of object-classes you need the image vector will be extended. For our purpose again it is not necessary.  

In [ ]:
def testGenerator(test_path, image_color_mode="grayscale"):

    image_generator = image_dataset_from_directory(
        test_path,
        labels=None,
        label_mode="binary",
        color_mode=image_color_mode,
        image_size=target_size,
        batch_size=2,
        shuffle=False,
    ).map(lambda x: (rescale_data(x)))

    return image_generator

#### The visualizer of the labels and predictions

In [ ]:
def labelVisualize(num_class, img):
    img = img[:, :, 0] if len(img.shape) == 3 else img
    img_out = np.zeros(img.shape + (3,))
    for i in range(num_class):
        img_out[img == i, :] = color_dict[i]
    return img_out / 255

#### Saving the results

In [ ]:
def saveResult(save_path, npyfile, num_class=2):
    for i, item in enumerate(npyfile):
        img = np.array(item[:, :, 0] * 255, dtype=np.uint8)
        io.imsave(os.path.join(save_path, "%d_predict.png" % i), img)

## Run the training

Now all functions we need to run our Unet are defined and thus we can start the training.

First we need to augmentate our dataset which contains up to now only 30 images.
Therefor we can define some parameters within out data augmentation dictionary "data_gen_args" for this augmentation. To get some results and not just a gray image as prediciton, you should start with small parameters here.

Then we will generate the data with the beforehand defined parameters and the images given in the training folder. 
The first number hereby defines the batch_size, which you can change to a bigger number if your Computer fill in the requirements (GPU, Memory).

Now it is time to built our UNet with just calling the function and than start the traing.
You can change the "steps_per_epoch" and "epochs". The more you chose the better the results will get but also the longer the training will take.
The Checkpoint-file will be saved within your Jupter Notebook folder.

In [ ]:
##########################################################
#           In this box some code must be added          #
##########################################################
# if it fails to run the training on your tensorflow-gpu
# rerun it on a tensorflow cpu version

data_gen_args = dict(
    rotation_range=__,
    width_shift_range=__,
    height_shift_range=__,
    zoom_range=__,
    resize_height=256,
    resize_width=256,
    flip=__,
    interpolation="nearest",
)

myGene = trainGenerator(2, "data/membrane/train", "image", "label", data_gen_args)

model = unet()
model_checkpoint = ModelCheckpoint(
    "unet_membrane.hdf5", monitor="loss", verbose=1, save_best_only=True
)
model.fit(myGene, steps_per_epoch=300, epochs=1, callbacks=[model_checkpoint])

## Test your Model

When the training is done you can generate some prediction on your test data and save them within the test-image folder.

In [ ]:
testGene = testGenerator("data/membrane/test")
results = model.predict(testGene, 30, verbose=1)

if not os.path.exists("data/prediction"):
    os.mkdir("data/prediction")

saveResult("data/prediction", results)

### Show the results

We can now either look in the folder how well our network was trained or just show some images and their predicitons here:
If your prediction is just a gray image, your model wasn't trained good enough, and you might want to change some parameters (first of all steps_per_epoch and epochs)

In [ ]:
test_path = "data/membrane/test/"
predict_path = "data/prediction/"
# number 0 to 29 your want to show
img_number = 1
display(Image(test_path + str(img_number) + ".png"))
display(Image(predict_path + str(img_number) + "_predict.png"))

## What's next

Some ideas of what you can do next

- Try on cars 
- try on dataset with multiple labels 
- try on zebrafish repository (https://osf.io/snb6p/)
- adapt architecture (loss, learning-rate,...)

## Sources:<br>
[Fig2 - https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/u-net-architecture.png](https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/u-net-architecture.png)<br>
[Fig3 - https://arxiv.org/pdf/1505.04597.pdf](https://arxiv.org/pdf/1505.04597.pdf) <br>